In [43]:
from collections import defaultdict
import os
import pickle
import random
import requests
import time
import tqdm

import scipy.sparse as sp

from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.utils.data as td
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam

from utils import (EvalDataset, OUNoise, Prioritized_Buffer, get_beta, 
                   preprocess_data, to_np, hit_metric, dcg_metric)

In [42]:
# df = pd.read_csv("data/beer_reviews.csv")
# df = df[["beer_beerid", "review_profilename","review_overall","review_aroma","review_appearance","review_palate","review_taste",]]
# df = df.dropna(axis=0)
#
# df["beer_id"] = df.beer_beerid.astype('category').cat.codes
# df["reviewer_id"] = df.review_profilename.astype('category').cat.codes
#
# df = df.drop("beer_beerid", axis=1)
# df = df.drop("review_profilename", axis=1)
#
# df = df.rename(columns={"review_overall":"overall",
#                    "review_aroma":"aroma",
#                    "review_appearance":"appearance",
#                    "review_palate":"palate",
#                    "review_taste":"taste"}
#           )
#
# df.to_csv("data/clean_beer_reviews.csv", index=False)

In [46]:
data_dir = "data"
rating = "clean_beer_reviews.csv"

params = {
    'batch_size': 512,
    'embedding_dim': 8,
    'hidden_dim': 16,
    'N': 5, # memory size for state_repr
    'ou_noise':False,
    
    'value_lr': 1e-5,
    'value_decay': 1e-4,
    'policy_lr': 1e-5,
    'policy_decay': 1e-6,
    'state_repr_lr': 1e-5,
    'state_repr_decay': 1e-3,
    'log_dir': 'logs/final/',
    'gamma': 0.8,
    'min_value': -10,
    'max_value': 10,
    'soft_tau': 1e-3,
    
    'buffer_size': 1000000
}

## 0. Problem statement

Traditional recommendation task can be treated as sequental desicion making problem.
Recommender (i.e. agent) interacts with users (i.e. environment) to sequentally suggest set of items.
The goal is to maximize clients' satisfaction (i.e. reward).
More specifically:
- State is a vector $a \in R^{3\cdot embedding\_dim}$ computed using the user embedding and the embeddings of `N` latest positive interactions. In the code (replay buffer) state is represented  by `(user, memory)`
- Action is a vector $a \in R^{embedding\_dim}$. To get ranking score we took dot product of
the action and the item embedding (similar to word2vec and other embedding models).
- Reward is taken from user-item matrix (1 if rating > 3, 0 otherwise)

Reinforcement Learning can help recommendation at least in 2 ways.
1. User’s preference on previous items will affect his choice on the next items. 
User tends to give a higher rating if he has consecutively received more satisfied items (and vice versa). 
So, it would be more reasonable to model the recommendation as a sequential decision making process.
2. It is important to use long-term planning in recommendations. For example, after reading the weather forecast, the user is not willing to read similar news. On the other hand, after watching funny videos or reading memes the user can constanly do the same.

In [44]:
def preprocess_milti_data(data_dir, train_rating):
    data = pd.read_csv(os.path.join(data_dir, train_rating))
    data = data[['reviewer_id', 'beer_id', "overall","aroma","appearance","palate","taste"]]

    user_num = data['reviewer_id'].max() + 1
    item_num = data['beer_id'].max() + 1

    train_data = data.sample(frac=0.8, random_state=16)
    test_data = data.drop(train_data.index)

    all_aspect_train_matrix = {}
    all_aspect_test_matrix = {}

    for aspect in ["overall","aroma","appearance","palate","taste"]:
        aspect_train_data = train_data[train_data[aspect] > 3][['reviewer_id', 'beer_id']].values.tolist()
        aspect_test_data = test_data[test_data[aspect] > 3][['reviewer_id', 'beer_id']].values.tolist()

        train_matrix = sp.dok_matrix((user_num, item_num), dtype=np.float32)
        for user, item in aspect_train_data:
            train_matrix[user, item] = 1.0
        test_matrix = sp.dok_matrix((user_num, item_num), dtype=np.float32)
        for user, item in aspect_test_data:
            test_matrix[user, item] = 1.0

        all_aspect_train_matrix[aspect] = train_matrix
        all_aspect_test_matrix[aspect] = test_matrix


    appropriate_users = np.arange(user_num).reshape(-1, 1)[(all_aspect_train_matrix["overall"].sum(1) >= 20)]

    return all_aspect_train_matrix, all_aspect_test_matrix, user_num, item_num, appropriate_users

In [ ]:
aspect_num = 5

In [48]:
all_aspect_train_matrix, all_aspect_test_matrix, user_num, item_num, appropriate_users\
    = preprocess_milti_data(data_dir, rating)

KeyboardInterrupt: 

## 1. Environment

- **Observation space**. As mentioned before, to get state we need `N` latest positive items (`memory`) and embedding of user. `State_Repr_Module` transform it to the vector of dimensionality `embedding_dim * 3`.

- **Action space**. For every user we sample nonrelated items (the same count as related). All `available_items` which wasn't viewed before form action space.

Given a state we get action embedding, compute dot product between this embedding and embeddings of all items in action space, take 1 top ranked item, compute reward, update `viewed_items` and memory, and store transition in buffer.

In [59]:
class Env():
    def __init__(self, user_item_matrix):
        self.matrix = user_item_matrix
        self.item_count = item_num
        self.memory = np.ones([user_num, params['N']]) * item_num
        # memory is initialized as [item_num] * N for each user
        # it is padding indexes in state_repr and will result in zero embeddings

    def reset(self, user_id):
        self.user_id = user_id
        self.viewed_items = []
        self.related_items = np.argwhere(self.matrix[self.user_id] > 0)[:, 1]
        self.num_rele = len(self.related_items)
        self.nonrelated_items = np.random.choice(
            list(set(range(self.item_count)) - set(self.related_items)), self.num_rele)
        self.available_items = np.zeros(self.num_rele * 2)
        self.available_items[::2] = self.related_items
        self.available_items[1::2] = self.nonrelated_items
        
        return torch.tensor([self.user_id]), torch.tensor(self.memory[[self.user_id], :])
    
    def step(self, action, action_emb=None, buffer=None):
        initial_user = self.user_id
        initial_memory = self.memory[[initial_user], :]
        
        reward = float(to_np(action)[0] in self.related_items)
        self.viewed_items.append(to_np(action)[0])
        if reward:
            if len(action) == 1:
                self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]
            else:
                self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action[0]]
                
        if len(self.viewed_items) == len(self.related_items):
            done = 1
        else:
            done = 0
            
        if buffer is not None:
            buffer.push(np.array([initial_user]), np.array(initial_memory), to_np(action_emb)[0], 
                        np.array([reward]), np.array([self.user_id]), self.memory[[self.user_id], :], np.array([reward]))

        return torch.tensor([self.user_id]), torch.tensor(self.memory[[self.user_id], :]), reward, done

## 2. Model

### Overall model

<img src="img/full_model.png" width="1000" height="700">

In [9]:
class Actor_DRR(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
    
        self.layers = nn.Sequential(
            nn.Linear(embedding_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embedding_dim)
        )
        
        self.initialize()

    def initialize(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)

    def forward(self, state):
        return self.layers(state)
    
    def get_action(self, user, memory, state_repr, 
                   action_emb,
                   items=torch.tensor([i for i in range(item_num)]),
                   return_scores=False
                  ):
        state = state_repr(user, memory)
        scores = torch.bmm(state_repr.item_embeddings(items).unsqueeze(0), 
                         action_emb.T.unsqueeze(0)).squeeze(0)
        if return_scores:
            return scores, torch.gather(items, 0, scores.argmax(0))
        else:
            return torch.gather(items, 0, scores.argmax(0))

In [10]:
class Critic_DRR(nn.Module):
    def __init__(self, state_repr_dim, action_emb_dim, hidden_dim):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(state_repr_dim + action_emb_dim, hidden_dim), 
            nn.ReLU(), 
            nn.Linear(hidden_dim, 1)
        )

        self.initialize()
        
    def initialize(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = self.layers(x)
        return x

### State representation

<img src="img/state_representation.png" width="1400" height="1000">

In [11]:
class State_Repr_Module(nn.Module):
    def __init__(self, user_num, item_num, embedding_dim, hidden_dim):
        super().__init__()
        self.user_embeddings = nn.Embedding(user_num, embedding_dim)
        self.item_embeddings = nn.Embedding(item_num+1, embedding_dim, padding_idx=int(item_num))
        self.drr_ave = torch.nn.Conv1d(in_channels=params['N'], out_channels=1, kernel_size=1)
        
        self.initialize()
            
    def initialize(self):
        nn.init.normal_(self.user_embeddings.weight, std=0.01)
        nn.init.normal_(self.item_embeddings.weight, std=0.01)
        self.item_embeddings.weight.data[-1].zero_()
        nn.init.uniform_(self.drr_ave.weight)
        self.drr_ave.bias.data.zero_()

    def forward(self, user, memory):
        user_embedding = self.user_embeddings(user.long())

        item_embeddings = self.item_embeddings(memory.long())
        drr_ave = self.drr_ave(item_embeddings).squeeze(1)
        
        return torch.cat((user_embedding, user_embedding * drr_ave, drr_ave), 1)

For evaluation we take 1 positive and 99 sampled negatives items per batch, select 10 items with best scores and calculate hit_rate@10 and nDCG@10.
During training we choose user 6039 and track `hit` and `dcg` only for him (for evaluation speed). Final scores was computed on the whole test data.

In [12]:
valid_dataset = EvalDataset(
    np.array(test_data)[np.array(test_data)[:, 0] == 6039], 
    item_num, 
    test_matrix)
valid_loader = td.DataLoader(valid_dataset, batch_size=100, shuffle=False)

full_dataset = EvalDataset(np.array(test_data), item_num, test_matrix)
full_loader = td.DataLoader(full_dataset, batch_size=100, shuffle=False)

Resetting dataset
Resetting dataset


In [13]:
def run_evaluation(net, state_representation, training_env_memory, loader=valid_loader):
    hits = []
    dcgs = []
    test_env = Env(test_matrix)
    test_env.memory = training_env_memory.copy()
    user, memory = test_env.reset(int(to_np(next(iter(valid_loader))['user'])[0]))
    for batch in loader:
        action_emb = net(state_repr(user, memory))
        scores, action = net.get_action(
            batch['user'], 
            torch.tensor(test_env.memory[to_np(batch['user']).astype(int), :]), 
            state_representation, 
            action_emb,
            batch['item'].long(), 
            return_scores=True
        )
        user, memory, reward, done = test_env.step(action)

        _, ind = scores[:, 0].topk(10)
        predictions = torch.take(batch['item'], ind).cpu().numpy().tolist()
        actual = batch['item'][0].item()
        hits.append(hit_metric(predictions, actual))
        dcgs.append(dcg_metric(predictions, actual))
        
    return np.mean(hits), np.mean(dcgs)

## 3. Training

In [23]:
torch.manual_seed(2)

state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim'])
policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])
value_net  = Critic_DRR(params['embedding_dim'] * 3, params['embedding_dim'], params['hidden_dim'])
replay_buffer = Prioritized_Buffer(params['buffer_size'])

target_value_net  = Critic_DRR(params['embedding_dim'] * 3, params['embedding_dim'], params['hidden_dim'])
target_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)

value_criterion  = nn.MSELoss()
value_optimizer  = Adam(value_net.parameters(),  lr=params['value_lr'],
                          weight_decay=params['value_decay'])
policy_optimizer = Adam(policy_net.parameters(), lr=params['policy_lr'],
                          weight_decay=params['policy_decay'])
state_repr_optimizer = Adam(state_repr.parameters(), lr=params['state_repr_lr'],
                              weight_decay=params['state_repr_decay'])

writer = SummaryWriter(log_dir=params['log_dir'])

In [24]:
def ddpg_update(training_env, 
                step=0,
                batch_size=params['batch_size'], 
                gamma=params['gamma'],
                min_value=params['min_value'],
                max_value=params['max_value'],
                soft_tau=params['soft_tau'],
               ):
    beta = get_beta(step)
    user, memory, action, reward, next_user, next_memory, done = replay_buffer.sample(batch_size, beta)
    user        = torch.FloatTensor(user)
    memory      = torch.FloatTensor(memory)
    action      = torch.FloatTensor(action)
    reward      = torch.FloatTensor(reward)
    next_user   = torch.FloatTensor(next_user)
    next_memory = torch.FloatTensor(next_memory)
    done = torch.FloatTensor(done)
    
    state       = state_repr(user, memory)
    policy_loss = value_net(state, policy_net(state))
    policy_loss = -policy_loss.mean()
    
    next_state     = state_repr(next_user, next_memory)
    next_action    = target_policy_net(next_state)
    target_value   = target_value_net(next_state, next_action.detach())
    expected_value = reward + (1.0 - done) * gamma * target_value
    expected_value = torch.clamp(expected_value, min_value, max_value)

    value = value_net(state, action)
    value_loss = value_criterion(value, expected_value.detach())
    
    state_repr_optimizer.zero_grad()
    policy_optimizer.zero_grad()
    policy_loss.backward(retain_graph=True)
    policy_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward(retain_graph=True)
    value_optimizer.step()
    state_repr_optimizer.step()

    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
                target_param.data.copy_(
                    target_param.data * (1.0 - soft_tau) + param.data * soft_tau
                )

    for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

    writer.add_histogram('value', value, step)
    writer.add_histogram('target_value', target_value, step)
    writer.add_histogram('expected_value', expected_value, step)
    writer.add_histogram('policy_loss', policy_loss, step)

In [62]:
np.random.seed(16)
train_env = Env(train_matrix)
hits, dcgs = [], []
hits_all, dcgs_all = [], []
step, best_step = 0, 0
step, best_step, best_step_all = 0, 0, 0
users = np.random.permutation(appropriate_users)[:10]
ou_noise = OUNoise(params['embedding_dim'], decay_period=10)

eval_freq = 100
all_eval_freq = 10000

for u in tqdm.tqdm(users):
    user, memory = train_env.reset(u)
    if params['ou_noise']:
        ou_noise.reset()
    for t in range(int(train_matrix[u].sum())):
        action_emb = policy_net(state_repr(user, memory))
        if params['ou_noise']:
            action_emb = ou_noise.get_action(action_emb.detach().cpu().numpy()[0], t)
        action = policy_net.get_action(
            user, 
            torch.tensor(train_env.memory[to_np(user).astype(int), :]), 
            state_repr, 
            action_emb,
            torch.tensor(
                [item for item in train_env.available_items 
                if item not in train_env.viewed_items]
            ).long()
        )
        user, memory, reward, done = train_env.step(
            action, 
            action_emb,
            buffer=replay_buffer
        )

        if len(replay_buffer) > params['batch_size']:
            ddpg_update(train_env, step=step)

        if step % eval_freq == 0 and step > 0:
            hit, dcg = run_evaluation(policy_net, state_repr, train_env.memory)
            writer.add_scalar('hit', hit, step)
            writer.add_scalar('dcg', dcg, step)
            hits.append(hit)
            dcgs.append(dcg)
            if np.mean(np.array([hit, dcg]) - np.array([hits[best_step], dcgs[best_step]])) > 0:
                best_step = step // eval_freq
                torch.save(policy_net.state_dict(), params['log_dir'] + 'policy_net.pth')
                torch.save(value_net.state_dict(), params['log_dir'] + 'value_net.pth')
                torch.save(state_repr.state_dict(), params['log_dir'] + 'state_repr.pth')
        if step % all_eval_freq == 0 and step > 0:
            hit, dcg = run_evaluation(policy_net, state_repr, train_env.memory, full_loader)
            writer.add_scalar('hit_all', hit, step)
            writer.add_scalar('dcg_all', dcg, step)
            hits_all.append(hit)
            dcgs_all.append(dcg)
            if np.mean(np.array([hit, dcg]) - np.array([hits_all[best_step_all], dcgs_all[best_step_all]])) > 0:
                best_step_all = step // all_eval_freq
                torch.save(policy_net.state_dict(), params['log_dir'] + 'best_policy_net.pth')
                torch.save(value_net.state_dict(), params['log_dir'] + 'best_value_net.pth')
                torch.save(state_repr.state_dict(), params['log_dir'] + 'best_state_repr.pth')
        step += 1

  0%|          | 0/10 [00:00<?, ?it/s]/var/folders/rp/52dczj196xsgxn2_m6kh58_80000gn/T/ipykernel_14950/3553351555.py:30: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  self.memory[self.user_id] = list(self.memory[self.user_id][1:]) + [action]
100%|██████████| 10/10 [00:32<00:00,  3.29s/it]


In [65]:
torch.save(policy_net.state_dict(), params['log_dir'] + 'policy_net_final.pth')
torch.save(value_net.state_dict(), params['log_dir'] + 'value_net_final.pth')
torch.save(state_repr.state_dict(), params['log_dir'] + 'state_repr_final.pth')

In [20]:
# we need memory for validation, so it's better to save it and not wait next time 
with open('logs/memory.pickle', 'wb') as f:
    pickle.dump(train_env.memory, f)
    
with open('logs/memory.pickle', 'rb') as f:
    memory = pickle.load(f)

## 4. Results

Weights and logs are stored in [this folder](https://drive.google.com/drive/folders/1oz6w4HBEVspvReBQ55zeIE4qw5OPD84U?usp=sharing)

In [22]:
no_ou_state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim'])
no_ou_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])
no_ou_state_repr.load_state_dict(torch.load('logs/no_ou/' + 'best_state_repr.pth'))
no_ou_policy_net.load_state_dict(torch.load('logs/no_ou/' + 'best_policy_net.pth'))

hit, dcg = run_evaluation(no_ou_policy_net, no_ou_state_repr, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

hit rate:  0.45371301632835115 dcg:  0.25425667195713686


In [23]:
ou_state_repr = State_Repr_Module(user_num, item_num, params['embedding_dim'], params['hidden_dim'])
ou_policy_net = Actor_DRR(params['embedding_dim'], params['hidden_dim'])
ou_state_repr.load_state_dict(torch.load('logs/ou_noise_04/' + 'best_state_repr.pth'))
ou_policy_net.load_state_dict(torch.load('logs/ou_noise_04/' + 'best_policy_net.pth'))

hit, dcg = run_evaluation(ou_policy_net, ou_state_repr, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

hit rate:  0.5024706798086426 dcg:  0.28013841397710176


### Example of trained agents behaviour

Let's choose random user

In [67]:
random_user = np.random.randint(user_num)
print(random_user)

5537


In [71]:
movies = pd.read_csv('data/movies.dat', sep='::', header=None, engine='python', names=['id', 'name', 'genre'], encoding="")
# in the code numeration starts with 0
movies[movies['id'].isin(np.argwhere(test_matrix[random_user] > 0)[:, 1] + 1)]

,id,name,genre
26,27,Now and Then (1995),Drama
54,55,Georgia (1995),Drama
77,78,"Crossing Guard, The (1995)",Drama
133,135,Down Periscope (1996),Comedy
199,201,Three Wishes (1995),Drama
209,211,"Browning Version, The (1994)",Drama
607,611,Hellraiser: Bloodline (1996),Action|Horror|Sci-Fi
640,645,Nelly & Monsieur Arnaud (1995),Drama
664,670,"World of Apu, The (Apur Sansar) (1959)",Drama
748,758,"Jar, The (Khomreh) (1992)",Drama


For example we can recommend "Nixon" and "Love Serenade" and see next 3 predictions.

In [72]:
predictions = []

for model, state_representation in zip([ou_policy_net, no_ou_policy_net], [ou_state_repr, no_ou_state_repr]):
    example_env = Env(test_matrix)
    user, memory = example_env.reset(random_user)

    user, memory, reward, _ = example_env.step(torch.tensor([13]))
    user, memory, reward, _ = example_env.step(torch.tensor([1584]))
    preds = []
    for _ in range(3):
        action_emb = model(state_representation(user, memory))
        action = model.get_action(
            user, 
            torch.tensor(example_env.memory[to_np(user).astype(int), :]), 
            state_representation, 
            action_emb,
            torch.tensor(
                [item for item in example_env.available_items 
                if item not in example_env.viewed_items]
            ).long()
        )
        user, memory, reward, _ = example_env.step(action)
        preds.append(action)

    predictions.append(preds)

print(predictions[0])
print(predictions[1])

NameError: name 'ou_policy_net' is not defined

Model trained with OU noise recommended related `Comedy` and `Documentary` after that switch recommendations to nonrelated `Crime|Film-Noir|Thriller`.
Model trained without OU noise recommended `Comedy|Drama`, `Children's|Comedy`, `Drama` (two of them are related).

Both models seems to be reasonable.

### Training process logs

<img src=img/learning_curve.png>

Logs are consistent with expectations. Adding noise increase metrics (std=0.4 performs the best, after 0.6 model starts to degrade).